# Causal Modelling with DoWhy and EconML

# Imports

In [14]:
import numpy as np
import pandas as pd

from dowhy import CausalModel, plotter
from IPython.display import Image, display
import matplotlib.pyplot as plt

from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor

import econml
from econml.dml import LinearDML, CausalForestDML
from econml.cate_interpreter import SingleTreeCateInterpreter

# Read data

In [2]:
df = pd.read_csv('../data/causal_df.csv')

In [3]:
df.head()

,Unnamed: 0,date,day_of_week,weather_region_cd,maximum_feels_like_temperature,rainfall,sunshine_td,avg_ice_cream_price,ice_cream_sold,cpih,month
0,0,2022-03-27,0.0,1.0,15.4,0.2,12.1,2.704702,1498.0,6.2,3.0
1,1,2022-03-27,0.0,10.0,12.7,0.0,10.6,2.704702,2285.0,6.2,3.0
2,2,2022-03-27,0.0,11.0,15.1,0.0,10.2,2.704702,5509.0,6.2,3.0
3,3,2022-03-27,0.0,12.0,10.7,0.0,5.3,2.704702,7776.0,6.2,3.0
4,4,2022-03-27,0.0,13.0,13.1,0.0,8.6,2.704702,4903.0,6.2,3.0


# Creating Causal Graph

In [39]:
# Define estimator inputs
Y = df["ice_cream_sold"]  # outcome of interest
T = df["maximum_feels_like_temperature"]  # intervention, or treatment
X = df[["weather_region_cd", "day_of_week"]]  # features
W = df[["rainfall", "sunshine_td", "avg_ice_cream_price", "cpih", "month"]]  # confounders

# Identify Causal Effects

In [31]:
# Train EconML model
model = CausalForestDML(
    model_y=GradientBoostingRegressor(), model_t=GradientBoostingRegressor()
)

model = model.fit(Y, T, X=X, inference="blb")

In [32]:
model

# Estimate Causal Effects

In [38]:
intrp = SingleTreeCateInterpreter(include_model_uncertainty=False, max_depth=2, min_samples_leaf=1)
intrp.interpret(model, X)
plt.figure(figsize=(25, 5))
intrp.plot(feature_names=X.columns, fontsize=12)

# Refutes

In [21]:
refutel_random_common_cause = model.refute_estimate(estimands,estimate, "random_common_cause")

refutel_placebo_treatment_refuter = model.refute_estimate(estimands,estimate, "placebo_treatment_refuter")

refutel_data_subset_refuter = model.refute_estimate(estimands,estimate, "data_subset_refuter")

res_unobserved_range = model.refute_estimate(estimands, estimate, method_name="add_unobserved_common_cause",
                                           confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                           effect_strength_on_treatment=[0.001, 0.005, 0.01, 0.02],
                                           effect_strength_on_outcome=[0.001, 0.005, 0.01,0.02])


AttributeError: 'CausalForestDML' object has no attribute 'refute_estimate'

In [14]:
print(refutel_random_common_cause)

Refute: Add a random common cause
Estimated effect:528.9312108028014
New effect:526.5991101317206
p value:0.4



In [15]:
print(refutel_placebo_treatment_refuter)

Refute: Use a Placebo Treatment
Estimated effect:528.9312108028014
New effect:0.0
p value:1.0



In [16]:
print(refutel_data_subset_refuter)

Refute: Use a subset of data
Estimated effect:528.9312108028014
New effect:518.0634369834922
p value:0.25



In [17]:
print(res_unobserved_range)

Refute: Add an Unobserved Common Cause
Estimated effect:528.9312108028014
New effect:(51.15071819432224, 488.6751314859039)



# Final Estimates